# [Day 11: Monkey in the Middle](https://adventofcode.com/2022/day/11)

In [1]:
example = """Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
  Starting items: 54, 65, 75, 74
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 0

Monkey 2:
  Starting items: 79, 60, 97
  Operation: new = old * old
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 3

Monkey 3:
  Starting items: 74
  Operation: new = old + 3
  Test: divisible by 17
    If true: throw to monkey 0
    If false: throw to monkey 1"""

Parse monkey business from input.

In [21]:
from collections import namedtuple

Monkey = namedtuple('Monkey', 'items,op,divisor,true,false')

def parse(input):
    """Returns monkey attributes parsed from input."""
    monkeys = []
    for _, items, op, divisor, true, false in [chunk.splitlines() for chunk in input.split('\n\n')]:
        monkeys.append(Monkey(
            # Convert worry levels to integers.
            [int(item) for item in items.split(': ')[1].split(', ')],
            # Make the operation a lambda function - neat!
            eval('lambda old:' + op.split('=')[1]),
            # Extract the divisor - assuming it's always division.
            int(divisor.split()[-1]),
            # Destination monkey if true.
            int(true.split()[-1]),
            # Destination monkey if false.
            int(false.split()[-1])
        ))
    return monkeys

parse(example)

[Monkey(items=[79, 98], op=<function <lambda> at 0x10dbf6b60>, divisor=23, true=2, false=3),
 Monkey(items=[54, 65, 75, 74], op=<function <lambda> at 0x10dbf6c00>, divisor=19, true=2, false=0),
 Monkey(items=[79, 60, 97], op=<function <lambda> at 0x10dbf6ca0>, divisor=13, true=1, false=3),
 Monkey(items=[74], op=<function <lambda> at 0x10dbf6d40>, divisor=17, true=0, false=1)]

Play a round.

In [48]:
def play_round(monkeys, inspections, post_inspection_divisor=3):
    for index, monkey in enumerate(monkeys):
        # Inspect each item.
        while monkey.items:
            # Increment monkey's inspection count.
            inspections[index] += 1
            # Calculate new worry level.
            new = monkey.op(monkey.items.pop(0)) // post_inspection_divisor
            # Decide where to throw the item.
            if not new % monkey.divisor:
                monkeys[monkey.true].items.append(new)
            else:
                monkeys[monkey.false].items.append(new)
    return monkeys, inspections

play_round(monkeys := parse(example), [0] * len(monkeys))

([Monkey(items=[20, 23, 27, 26], op=<function <lambda> at 0x10dbf7ba0>, divisor=23, true=2, false=3),
  Monkey(items=[2080, 25, 167, 207, 401, 1046], op=<function <lambda> at 0x10dda7e20>, divisor=19, true=2, false=0),
  Monkey(items=[], op=<function <lambda> at 0x10dda7740>, divisor=13, true=1, false=3),
  Monkey(items=[], op=<function <lambda> at 0x10dda7240>, divisor=17, true=0, false=1)],
 [2, 4, 3, 5])

Calulate level of monkey business after 20 rounds.

In [49]:
import math

def calculate_monkey_business(monkeys, rounds=20):
    inspections = [0] * len(monkeys)
    for _ in range(rounds):
        monkeys, inspections = play_round(monkeys, inspections)
    # Multiply the inspection count of the two most active monkeys.
    return math.prod(sorted(inspections)[-2:])

calculate_monkey_business(parse(example))

10605

# Part 1

Calulate level of monkey business after 20 rounds using input.

In [50]:
calculate_monkey_business(parse(open('day-11-input.txt').read()))

62491

# Part 2

## Hmm I need a more efficient algorithm

Calculate monkey business after 10000 rounds, without dividing worry levels.

In [57]:
def calculate_monkey_business(monkeys, rounds=800):
    inspections = [0] * len(monkeys)
    for _ in range(rounds):
        monkeys, inspections = play_round(monkeys, inspections, post_inspection_divisor=1)
    # Multiply the inspection count of the two most active monkeys.
#     return math.prod(sorted(inspections)[-2:])
    return inspections

calculate_monkey_business(parse(example))

[4159, 3837, 162, 4149]